# 1. 原始方法

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


template = ChatPromptTemplate([
    ("system", """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation: """),
    ("placeholder", "{history}"),
    ("user", "{input}"),
    ("ai", "")
])


chat = ChatOpenAI(model_name="openai/gpt-4o-mini")
chain = template | chat | StrOutputParser()

resp = chain.invoke({
    "history": [
        ("human", "我周五晚上要去看电影"),
    ],
    "input": "我周五要去做什么"
})

print(resp)


你提到你周五晚上要去看电影，所以可以推测你周五会去电影院享受一部电影！你已经决定看哪部电影了吗？或者你是跟朋友一起去吗？
